# Import Statements

In [1]:
import datetime
import base64
from urllib.parse import urlencode
import json
import os

import requests

# Spotify WebAPI Client

Now that we have the Billboard Hot 100 data, I need to get metadata on each song that would be used to qualitatively and quantitatively describe the song. Spotify has this data available through their WebAPI in three formats. There is the basic information for each track (name, artist, length, popularity etc.), then along with this, each track has track features (danceability, energy, loudness, acousticness, etc.) and track analysis (end of fade in, start of fade out, tempo, key, mode). 

I will use a Spotify client to search the WebAPI for each song in the Hot100 dataset, and pull all basic info, features, and analysis into one new dataset which will be used for my recommendation engine. 

*I have set the client_id and client_secret as optional variables and have my application's authorizations saved as environment variables for security purposes. This code will work when run in my environment, however should crash for anyone who hasn't setup the same environment variables.*

In [20]:
# Source: https://www.youtube.com/watch?v=xdq6Gz33khQ&t=184s

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    
    
    def __init__(self, client_id=os.environ['HOT100_REC_CID'],\
                 client_secret=os.environ['HOT100_REC_CS'], *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
    'Authorization': f'Basic {client_creds_b64}'
               }

    def get_token_data(self):
        return {
    'grant_type': 'client_credentials'
               }
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        auth_done = self.perform_auth()
        if not auth_done:
            raise Exception("Authentication failed")
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
                  }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def get_track(self, _id):
        return self.get_resource(_id, resource_type='tracks')
    
    def get_features(self, _id):
        return self.get_resource(_id, resource_type='audio-features')
    
    def get_analysis(self, _id):
        return self.get_resource(_id, resource_type='audio-analysis')
    
    def base_search(self, query_params):
        access_token = self.get_access_token()
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

# Spotify Client Search Test

### Searching for song "Misery Business" by artist "Paramore" on album "Riot!"

In [14]:
spotify = SpotifyAPI()

In [15]:
spotify.search({"track": "Misery Business"}, search_type="track")

q=track%3AMisery+Business&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AMisery+Business&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/74XFHRwlV6OrjEM0A2NCMF'},
       'href': 'https://api.spotify.com/v1/artists/74XFHRwlV6OrjEM0A2NCMF',
       'id': '74XFHRwlV6OrjEM0A2NCMF',
       'name': 'Paramore',
       'type': 'artist',
       'uri': 'spotify:artist:74XFHRwlV6OrjEM0A2NCMF'}],
     'available_markets': ['BR', 'CA', 'US'],
     'external_urls': {'spotify': 'https://open.spotify.com/album/71rziY9eLo1tA2dBMxrwhc'},
     'href': 'https://api.spotify.com/v1/albums/71rziY9eLo1tA2dBMxrwhc',
     'id': '71rziY9eLo1tA2dBMxrwhc',
     'images': [{'height': 640,
       'url': 'https://i.scdn.co/image/ab67616d0000b273bee754528c08d5ff6799a1eb',
       'width': 640},
      {'height': 300,
       'url': 'https://i.scdn.co/image/ab67616d00001e02bee754528c08d5ff6799a1eb',
       'width': 300}

In [16]:
spotify.get_artist("74XFHRwlV6OrjEM0A2NCMF")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/74XFHRwlV6OrjEM0A2NCMF'},
 'followers': {'href': None, 'total': 6373444},
 'genres': ['candy pop', 'pixie', 'pop emo', 'pop punk'],
 'href': 'https://api.spotify.com/v1/artists/74XFHRwlV6OrjEM0A2NCMF',
 'id': '74XFHRwlV6OrjEM0A2NCMF',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb7052afa024fc4297bb55d483',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab676161000051747052afa024fc4297bb55d483',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f1787052afa024fc4297bb55d483',
   'width': 160}],
 'name': 'Paramore',
 'popularity': 79,
 'type': 'artist',
 'uri': 'spotify:artist:74XFHRwlV6OrjEM0A2NCMF'}

In [6]:
spotify.get_album("71rziY9eLo1tA2dBMxrwhc")

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/74XFHRwlV6OrjEM0A2NCMF'},
   'href': 'https://api.spotify.com/v1/artists/74XFHRwlV6OrjEM0A2NCMF',
   'id': '74XFHRwlV6OrjEM0A2NCMF',
   'name': 'Paramore',
   'type': 'artist',
   'uri': 'spotify:artist:74XFHRwlV6OrjEM0A2NCMF'}],
 'available_markets': ['BR', 'CA', 'US'],
 'copyrights': [{'text': '2007 Atlantic Recording Corporation for the United States and WEA International Inc. for the world outside of the United States. All Rights Reserved.',
   'type': 'C'},
  {'text': '2007 Atlantic Recording Corporation for the United States and WEA International Inc. for the world outside of the United States. All Rights Reserved.',
   'type': 'P'}],
 'external_ids': {'upc': '075679997449'},
 'external_urls': {'spotify': 'https://open.spotify.com/album/71rziY9eLo1tA2dBMxrwhc'},
 'genres': [],
 'href': 'https://api.spotify.com/v1/albums/71rziY9eLo1tA2dBMxrwhc',
 'id': '71rziY9eLo1tA2dBMxrwhc',
 'im

In [17]:
spotify.get_track("6SpLc7EXZIPpy0sVko0aoU")

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/74XFHRwlV6OrjEM0A2NCMF'},
    'href': 'https://api.spotify.com/v1/artists/74XFHRwlV6OrjEM0A2NCMF',
    'id': '74XFHRwlV6OrjEM0A2NCMF',
    'name': 'Paramore',
    'type': 'artist',
    'uri': 'spotify:artist:74XFHRwlV6OrjEM0A2NCMF'}],
  'available_markets': ['BR', 'CA', 'US'],
  'external_urls': {'spotify': 'https://open.spotify.com/album/71rziY9eLo1tA2dBMxrwhc'},
  'href': 'https://api.spotify.com/v1/albums/71rziY9eLo1tA2dBMxrwhc',
  'id': '71rziY9eLo1tA2dBMxrwhc',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab67616d0000b273bee754528c08d5ff6799a1eb',
    'width': 640},
   {'height': 300,
    'url': 'https://i.scdn.co/image/ab67616d00001e02bee754528c08d5ff6799a1eb',
    'width': 300},
   {'height': 64,
    'url': 'https://i.scdn.co/image/ab67616d00004851bee754528c08d5ff6799a1eb',
    'width': 64}],
  'name': 'Riot!',
  'release_date': '2007-06-11',
  'rele

In [8]:
spotify.get_features("6SpLc7EXZIPpy0sVko0aoU")

{'danceability': 0.537,
 'energy': 0.927,
 'key': 6,
 'loudness': -3.322,
 'mode': 1,
 'speechiness': 0.0476,
 'acousticness': 0.00017,
 'instrumentalness': 6.78e-06,
 'liveness': 0.115,
 'valence': 0.379,
 'tempo': 102.445,
 'type': 'audio_features',
 'id': '2PDLmgDWiwjUTxl1pPyDQA',
 'uri': 'spotify:track:2PDLmgDWiwjUTxl1pPyDQA',
 'track_href': 'https://api.spotify.com/v1/tracks/2PDLmgDWiwjUTxl1pPyDQA',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2PDLmgDWiwjUTxl1pPyDQA',
 'duration_ms': 228293,
 'time_signature': 4}

In [18]:
spotify.get_analysis("6SpLc7EXZIPpy0sVko0aoU")

{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1444722009,
  'analysis_time': 9.50884,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 4664016,
  'duration': 211.52,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 0.10503,
  'start_of_fade_out': 206.56471,
  'loudness': -3.677,
  'tempo': 172.977,
  'tempo_confidence': 0.501,
  'time_signature': 4,
  'time_signature_confidence': 1.0,
  'key': 1,
  'key_confidence': 0.248,
  'mode': 1,
  'mode_confidence': 0.405,
  'codestring': 'eJxNmgmS5DgOBL-iJ_A-_v-xdQ9mb82YtU2JUkokCAQiAO51Zh11r698vbW1eu_jW-Obp3LZyvjqKt_c_FVn61-b67urljnaLN843Dvj3tlX_dZuXq12al3f6e3bt_H2Nr9amm-qc57Ck7We8Z1bx9h9c9nH-O7Z3Jy7fHXwIi7XHoOP1MV8xl13lLv7V885vG_Xcmo_92uFj917GsuYTNDF7Mnfs_GlNgrTrLfWeed1-oWxxdNrtq-xLpfK0urgt2ecrxdujcm0enEeles6TsM-zGNfLjEa5hosafPaUjbP9cmrWHOr9z

# Conclusion

Lastly, I will export this class out to a .py file in order to import this class into all needed notebooks for this project. This file will be found at spotifyClient.py.